In [191]:
import numpy as np
from scipy.io.wavfile import read
from scipy import signal, arange
from scipy.signal import hilbert, find_peaks_cwt
import matplotlib.pyplot as plt
import matplotlib.axes as matax
import matplotlib.mlab as mlab
import sys
from math import exp
import operator

In [192]:
sampFreq7, unit7sampArray = read('unit7wav5.wav');
sampFreq8, unit8sampArray = read('unit8wav5.wav');
sampFreq13, unit13sampArray = read('unit13wav5.wav');
sampFreq20, unit20sampArray = read('unit20wav5.wav');

In [193]:
#sample frequency set equal to one of the sound files
#sound frequency since it should be the same for all
#files anyway
sampFreq = sampFreq7

#start time and duration measured in seconds
startTime = 0;
duration = duration = int(len(unit7sampArray)/sampFreq);

#in seconds
deadzoneInSeconds = 0.01

###DO NOT CHANGE THE NEXT TWO VARIABLES!!! 
#IF CHANGED PLEASE RESET leftOffset TO 50 AND rightOffset to 200
leftOffsetClassif = 50;  # how many frames we should expand the interval to the left (for the classfication algorithm)
rightOffsetClassif = 200; # how many frames we should expand ther interval to the right (for the classificaiton algorithm)
###DO NOT CHANGE THE LAST TWO VARIABLES!!!

#The deltaT (in seconds) for finding the same snap, for now I'll set it as some number that's 
#somewhat reasonable? idk....
deltaTinSeconds = 0.25;

#actual array indexes for splicing
startIndex = sampFreq * startTime;
endIndex = startIndex + sampFreq * duration;


unit7splicedArray = unit7sampArray[startIndex:endIndex];
unit8splicedArray = unit8sampArray[startIndex:endIndex];
unit13splicedArray = unit13sampArray[startIndex:endIndex];
unit20splicedArray = unit20sampArray[startIndex:endIndex];

In [194]:
#preparing 2 time arrays for the spliced signals for plotting (if needed) the x axis. 
#Indexes and seconds.
timeArrayInIndexes = np.arange(0, endIndex-startIndex, 1)
timeArrayInSeconds = timeArrayInIndexes / sampFreq

In [195]:
#takes in the sliced array and 
#returns an array of indexes and seconds where the amplitude
#is above the threshold. Already takes into account
#the small deadzone
def detectAboveThreshold(splicedArray):
    #setting up the amplitude threshold
    ampThreshold = np.std(splicedArray) * 10;
    
    #creates a boolean array for each index of the spiced sound array
    #each index of this boolean array corresponds to 1 index of the 
    #spliced sound array. The value will be true if above threshold,
    #false otherwise
    aboveThres = splicedArray > ampThreshold 
    
    #pick out the time when the amplitude is above threshold
    # current = the first time in seconds frame has amplitude above threshold
    # highAmplitudeSeconds = the time where each loud signal above threshold
    # highAmplitudeIndexes = the time frame indices of each loud signal above threshold
    highAmplitudeSeconds = [];
    highAmplitudeIndexes = [];
    current = timeArrayInSeconds[aboveThres][0] 
    
    #initializing the arrays with the first values before the loop 
    highAmplitudeSeconds.append(current)
    highAmplitudeIndexes.append(current * sampFreq)
    
    # set a deadzone (set to 0.01 seconds in the earlier cell) 
    # time window that slides across the current array
    # so that we only collect one time data for each loud signal
    for timeInSeconds in timeArrayInSeconds[aboveThres]: 
    
        if (timeInSeconds > current + deadzoneInSeconds ):
            highAmplitudeIndexes.append(timeInSeconds * sampFreq)
            highAmplitudeSeconds.append(timeInSeconds)
            current = timeInSeconds
            
    
    return highAmplitudeIndexes, highAmplitudeSeconds;
    

In [196]:
#grabs the indexes and times when amplitude goes above threshold
highAmpIndexesUnit7, highAmpSecondsUnit7 = detectAboveThreshold(unit7splicedArray)
highAmpIndexesUnit8, highAmpSecondsUnit8 = detectAboveThreshold(unit8splicedArray)
highAmpIndexesUnit13, highAmpSecondsUnit13 = detectAboveThreshold(unit13splicedArray)
highAmpIndexesUnit20, highAmpSecondsUnit20 = detectAboveThreshold(unit20splicedArray)

In [197]:
#a function that takes in the array of indexes of all signals (both snaps and pings)
#above threshold and outputs the array of indexes where SNAPS were found. 
def getSnapIndexes(splicedArray, highAmpIndexes):
    snapIndexes = [];
    
    for i in range(0, len(highAmpIndexes)):
        frontIndex = int(highAmpIndexes[i] - leftOffsetClassif);
        backIndex = int(highAmpIndexes[i] + rightOffsetClassif);
        
        signalForClassification = splicedArray[frontIndex:backIndex]
        classification = classify(signalForClassification)
        
        
        if(classification == "Snap"):
            snapIndexes.append(highAmpIndexes[i]);
        
    
    return snapIndexes;
            

In [198]:
def getFrequencies(signal):
    length = len(signal) # length of the signal
    #k = arange(length)
    #T = length/sampFreq
    #frq = k/T # two sides frequency range
    #frq = frq[range(np.int(length/2))] # one side frequency range

    frequencies = np.fft.fft(signal)/length # fft computing and normalization
    frequencies = frequencies[range(np.int(length/2))]
    
    return abs(frequencies)

In [199]:
#I did not change this snippet of code, it should be identical to the other one
def getRatio(signal):
    frequencies = getFrequencies(signal)
    ratio = sum(frequencies[34:68] * 100/sum(frequencies))
    return ratio;
    

In [200]:
#I did not change this snippet of code, it should be identical to the other one
def classify(signal):
    #print(len(signal))
    
    ratio = getRatio(signal)
    #print(ratio)
    if (ratio > 70):
        return "Ping"
    
    elif (ratio < 50):
        return "Snap"
    else:
        return "Undefined"

In [201]:
snapIndexesUnit7 = getSnapIndexes(unit7splicedArray, highAmpIndexesUnit7);
snapIndexesUnit8 = getSnapIndexes(unit8splicedArray, highAmpIndexesUnit8);
snapIndexesUnit13 = getSnapIndexes(unit13splicedArray, highAmpIndexesUnit13);
snapIndexesUnit20 = getSnapIndexes(unit20splicedArray, highAmpIndexesUnit20);

In [202]:
#test
print(len(snapIndexesUnit13))

print(len(highAmpIndexesUnit13))


849
1543


In [203]:
#TODO: function that would look at the snapIndexes of the 4 files and 
#group the snaps

#preliminarily making deltaT to be 70ms = 0.07s (both sides??)
deltaTinIndexes =  int(0.07 * sampFreq)

print(deltaTinIndexes)


4585


In [204]:
def snapPresent(snapIndexes, timeIndex):
    for snapIndex in snapIndexes:
        #print (snapIndex, timeIndex)
        if(snapIndex - deltaTinIndexes > timeIndex):
            break
       
        frontIndex = snapIndex - deltaTinIndexes 
        backIndex = snapIndex + deltaTinIndexes
        
        if (frontIndex < timeIndex and timeIndex < backIndex ):
            return True
    
    return False

In [205]:
def snapPresentInAll(snapIndexesUnit8, snapIndexesUnit13, snapIndexesUnit20, snapIndex):
    presentIn8 = snapPresent(snapIndexesUnit8, snapIndex)
    presentIn13 = snapPresent(snapIndexesUnit13, snapIndex)
    presentIn20 = snapPresent(snapIndexesUnit20, snapIndex)
    
    #print(presentIn8, presentIn13, presentIn20)
    return snapPresent(snapIndexesUnit8, snapIndex) and snapPresent(snapIndexesUnit13, snapIndex) and snapPresent(snapIndexesUnit20, snapIndex)
         

In [206]:
def plotAmplitude(snapIndex, ax, splicedArray):
    frontIndex = int(snapIndex - deltaTinIndexes)
    backIndex = int(snapIndex + deltaTinIndexes)
    
    print(backIndex)
    
    xAxis = timeArrayInSeconds[frontIndex-startIndex:backIndex+startIndex]
    yAxis = splicedArray[frontIndex:backIndex]
    
    ax.plot(xAxis, yAxis)
    ax.set_xlabel('time [Seconds]', fontsize=5)
    ax.set_ylabel('Amplitude', fontsize=5)
    ax.tick_params(axis = 'both', labelsize = 5)
    #ax.set_xlim(front, back)
    #ax.margins(x=0)

In [207]:
overlappingSnaps = [];
#note: did not discard overlaps yet
for snapIndex in snapIndexesUnit7:
    #print(snapIndex)
    if (snapPresentInAll(snapIndexesUnit8, snapIndexesUnit13, snapIndexesUnit20, snapIndex)):
        overlappingSnaps.append(snapIndex)

In [ ]:
len(overlappingSnaps)


344

In [ ]:
for i in range(0, len(overlappingSnaps)):
    fig, axes = plt.subplots(nrows=4, ncols=1)
    fig.suptitle("Snap number %d"%(i))
    #fig8, axes8 = plt.subplots(nrows=1, ncols=1)
    #fig13, axes13 = plt.subplots(nrows=1, ncols=1)
    #fig20, axes20 = plt.subplots(nrows=1, ncols=1)
    
    fname ='../Plots/CorrelatedSnaps/%d. %f-%f.png'% (i, (overlappingSnaps[i] - deltaTinIndexes)/sampFreq, (overlappingSnaps[i] + deltaTinIndexes)/sampFreq)
    #fname8 ='../Plots/CorrelatedSnaps/%d. Unit8: %f-%f.png'% (i, (overlappingSnaps[i] - deltaTinIndexes)/sampFreq, (overlappingSnaps[i] + deltaTinIndexes)/sampFreq)
    #fname13 ='../Plots/CorrelatedSnaps/%d. Unit13: %f-%f.png'% (i, (overlappingSnaps[i] - deltaTinIndexes)/sampFreq, (overlappingSnaps[i] + deltaTinIndexes)/sampFreq)
    #fname20 ='../Plots/CorrelatedSnaps/%d. Unit20: %f-%f.png'% (i, (overlappingSnaps[i] - deltaTinIndexes)/sampFreq, (overlappingSnaps[i] + deltaTinIndexes)/sampFreq)
    
    plotAmplitude(overlappingSnaps[i], axes[0], unit7splicedArray)
    plotAmplitude(overlappingSnaps[i], axes[1], unit8splicedArray)
    plotAmplitude(overlappingSnaps[i], axes[2], unit13splicedArray)
    plotAmplitude(overlappingSnaps[i], axes[3], unit20splicedArray)
    
    fig.savefig(fname, dpi = 300)
    #fig8.savefig(fname8)
    #fig13.savefig(fname13)
    #fig20.savefig(fname20)
    
    plt.close()
    
    

374948
374948
374948
374948
507180
507180
507180
507180
1707513
1707513
1707513
1707513
2031013
2031013
2031013
2031013
2238980
2238980
2238980
2238980
3285252
3285252
3285252
3285252
3422743
3422743
3422743
3422743
3591587
3591587
3591587
3591587
3654166
3654166
3654166
3654166
3807678
3807678
3807678
3807678
4500900
4500900
4500900
4500900
4508226
4508226
4508226
4508226
4747187
4747187
4747187
4747187
5604125
5604125
5604125
5604125
5741835
5741835
5741835
5741835
5743644
5743644
5743644
5743644
5856733
5856733
5856733
5856733
6026157
6026157
6026157
6026157
6869128
6869128
6869128
6869128
7099597
7099597
7099597
7099597
7241876
7241876
7241876
7241876
7374108
7374108
7374108
7374108
7477740
7477740
7477740
7477740
7490586
7490586
7490586
7490586
7695444
7695444
7695444
7695444
7718408
7718408
7718408
7718408
7984521
7984521
7984521
7984521
8069447
8069447
8069447
8069447
8083185
8083185
8083185
8083185
8314507
8314507
8314507
8314507
8655231
8655231
8655231
8655231
8699904
8699904


61227005
61227005
61227005
61227005
61230100
61230100
61230100
61230100
61525915
61525915
61525915
61525915
61527600
61527600
61527600
61527600
61760616
61760616
61760616
61760616
61927273
61927273
61927273
61927273
62127492
62127492
62127492
62127492
62129875
62129875
62129875
62129875
62140954
62140954
62140954
62140954
62231752
62231752
62231752
62231752
62380235
62380235
62380235
62380235
62503087
62503087
62503087
62503087
62535654
62535654
62535654
62535654
62537145
62537145
62537145
62537145
62921639
62921639
62921639
62921639
63062270
63062270
63062270
63062270
63146924
63146924
63146924
63146924
63166494
63166494
63166494
63166494
63283326
63283326
63283326
63283326
63557320
63557320
63557320
63557320
63558354
63558354
63558354
63558354
63598489
63598489
63598489
63598489
63863943
63863943
63863943
63863943
64255989
64255989
64255989
64255989
64263264
64263264
64263264
64263264
64421240
64421240
64421240
64421240
64447856
64447856
64447856
64447856
64740397
64740397
64740397
6